In [54]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("mines-paristech-cmm/mvtec-debug")

summary_list, config_list, name_list = [], [], []
tags_list = []
ids_list = []
for run in runs: 
    summary_list.append(run.summary._json_dict)
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})
    name_list.append(run.name)
    tags_list.append(run.tags)
    ids_list.append(run.id)
runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list,
    'tags': tags_list,
    'id': ids_list,
    })

In [55]:
runs_df.shape

(80, 5)

In [56]:
runs_df.columns

Index(['summary', 'config', 'name', 'tags', 'id'], dtype='object')

In [57]:
# unique tags in runs
set.union(*runs_df["tags"].apply(set).tolist())

{'dev01',
 'gpu-triple-run',
 'parallel-run-test',
 'run-n-parallel-1',
 'run-n-parallel-2',
 'run-n-parallel-3',
 'u2net',
 'u2net-tweaking',
 'u2net-tweaking-01'}

In [58]:
searched_tag = "parallel-run-test"
is_parallel_run_test = runs_df["tags"].apply(lambda tags: searched_tag in tags)
df = runs_df[is_parallel_run_test].copy(deep=True)
df.shape

(33, 5)

In [59]:
# unique tags in the SELECTED runs 
set.union(*df["tags"].apply(set).tolist())

{'dev01',
 'parallel-run-test',
 'run-n-parallel-1',
 'run-n-parallel-2',
 'run-n-parallel-3'}

In [60]:
removed_tags = ["dev01", "parallel-run-test"]
df["tags"] = df["tags"].apply(lambda tags: [tag for tag in tags if tag not in removed_tags][0])

In [68]:
df["nparallel"] = df["tags"].apply(lambda tag: tag.split("-")[-1]).apply(int)

In [72]:
import progressbar

for rowidx, row in progressbar.progressbar(df.iterrows(), max_value=df.shape[0]):
    run = api.run(f"mines-paristech-cmm/mvtec-debug/{row['id']}")
    run.config.update(dict(nparallel=row["nparallel"]))
    run.save()

100% (33 of 33) |########################| Elapsed Time: 0:03:43 Time:  0:03:43
